# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys2 import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kharkhorin,47.1975,102.8238,3.85,69,62,1.70,MN,1712525159
1,1,hawaiian paradise park,19.5933,-154.9731,23.82,91,75,5.14,US,1712525007
2,2,port-aux-francais,-49.3500,70.2167,4.67,70,100,3.05,TF,1712524903
3,3,mae hong son,19.3003,97.9685,24.16,63,2,0.27,TH,1712525163
4,4,waitangi,-43.9535,-176.5597,13.23,82,100,2.68,NZ,1712524994


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
# %%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.90,
    color = "City", alpha=0.7)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_ideal=city_data_df.loc[(city_data_df['Max Temp']>=20) & (city_data_df['Max Temp']<=30) & (city_data_df['Cloudiness']<=10) & (city_data_df['Wind Speed']<=2)] 

# Drop any rows with null values
city_data_df_ideal = city_data_df_ideal.dropna()

# Display sample data
city_data_df_ideal.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,mae hong son,19.3003,97.9685,24.16,63,2,0.27,TH,1712525163
63,63,walterboro,32.9052,-80.6668,22.25,28,0,0.00,US,1712525169
163,163,rio das ostras,-22.5269,-41.9450,25.95,88,1,1.95,BR,1712525050
178,178,karwar,14.8000,74.1333,26.60,75,7,1.82,IN,1712525180
274,274,nsanje,-16.9200,35.2620,23.09,95,10,1.26,MW,1712524933


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_ideal[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']= ""
hotel_df.reset_index(drop=True, inplace=True)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mae hong son,TH,19.3003,97.9685,63,
1,walterboro,US,32.9052,-80.6668,28,
2,rio das ostras,BR,-22.5269,-41.9450,88,
3,karwar,IN,14.8000,74.1333,75,
4,nsanje,MW,-16.9200,35.2620,95,
5,chemin grenier,MU,-20.4872,57.4656,90,
6,razole,IN,16.4833,81.8333,96,
7,shajapur,IN,23.4333,76.2667,23,
8,jeronimo monteiro,BR,-20.7894,-41.3950,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [57]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel","apiKey":geoapify_key}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Latitude=row['Lat']
    Longitude=row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mae hong son - nearest hotel: Jongkham Place
walterboro - nearest hotel: Travelodge
rio das ostras - nearest hotel: Pousada MariaMaria
karwar - nearest hotel: Emerald Bay, Karwar, Sterling Holidays
nsanje - nearest hotel: No hotel found
chemin grenier - nearest hotel: Hotel Shanti Maurice
razole - nearest hotel: No hotel found
shajapur - nearest hotel: No hotel found
jeronimo monteiro - nearest hotel: Hotel Kleyde


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mae hong son,TH,19.3003,97.9685,63,Jongkham Place
1,walterboro,US,32.9052,-80.6668,28,Travelodge
2,rio das ostras,BR,-22.5269,-41.9450,88,Pousada MariaMaria
3,karwar,IN,14.8000,74.1333,75,"Emerald Bay, Karwar, Sterling Holidays"
4,nsanje,MW,-16.9200,35.2620,95,No hotel found
5,chemin grenier,MU,-20.4872,57.4656,90,Hotel Shanti Maurice
6,razole,IN,16.4833,81.8333,96,No hotel found
7,shajapur,IN,23.4333,76.2667,23,No hotel found
8,jeronimo monteiro,BR,-20.7894,-41.3950,91,Hotel Kleyde


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [63]:
# %%capture --no-display
hotel_df_display = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df_display
# Configure the map plot
map_plot_2 = hotel_df_display.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.90,
    color = "City", alpha=0.7,
    hover_cols=["Hotel Name","Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)